<center>
<img src="../../img/ods_stickers.jpg">
## Open Machine Learning Course
<center>
Author: Yury Kashnitsky, Data Scientist at Mail.Ru Group

This material is subject to the terms and conditions of the license [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Free use is permitted for any non-comercial purpose with an obligatory indication of the names of the authors and of the source.

## <center>Assignment #6. Part 1
### <center> Beating benchmarks in "Catch Me If You Can: Intruder Detection through Webpage Session Tracking"
    
[Competition](https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2). The task is to beat "Assignment 6 baseline".

In [7]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import os
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, hstack, vstack
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.feature_extraction.text import TfidfVectorizer

Reading original data

In [8]:
PATH_TO_DATA = ('/Users/lucky/.kaggle/competitions/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2')
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'), index_col='session_id')
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'), index_col='session_id')

In [9]:
%%time

site_cols = ['site'+str(i) for i in range(1, 11)]
time_cols = ['time'+str(i) for i in range(1, 11)]

def prepare_dataset(df, is_test=False):
    
    train_sites = train_df[site_cols].fillna(0).astype(int)
    train_times = train_df[time_cols].fillna(method='ffill', axis=1).astype('datetime64[ns]')

    list_to_concat = [train_sites, train_times]
    if not is_test:
        list_to_concat.append(train_df['target'])
    df = pd.concat(list_to_concat, axis=1)

    df['#unique_sites'] = train_sites.nunique(axis=1) / 10 # to scale the feature we divide it by 10

    df['hour_of_day'] = df['time1'].dt.hour
    df['day_of_week'] = df['time1'].dt.dayofweek
    df['weekend'] = df['day_of_week'].apply(lambda d: 1 if d==5 or d==6 else 0)
    df['part_of_day'] = df['hour_of_day'].apply(lambda h: 1 if h > 11 and h <= 13 else (
                                                     2 if h > 15 and h <= 18 else (
                                                     3 if h > 18 and h <= 24 
                                                         else 4)))

    df['session_span'] = (df.time10 - df.time1).astype('timedelta64[s]')
    for i in range(1, 10):
        df['diff'+str(i)] = (df['time'+str(i+1)] - df['time'+str(i)]).astype('timedelta64[s]')
    
    return df
    
train_dataset = prepare_dataset(train_df)
test_dataset = prepare_dataset(test_df, is_test=True)

CPU times: user 48.6 s, sys: 830 ms, total: 49.4 s
Wall time: 49.3 s


Separate target feature 

In [10]:
y = train_df['target']

Build Tf-Idf features based on sites. You can use `ngram_range`=(1, 3) and `max_features`=100000 or more

In [11]:
%%time

train_sessions = train_dataset[site_cols].astype(str).apply(lambda s: ' '.join(s), axis=1)
test_sessions = test_dataset[site_cols].astype(str).apply(lambda s: ' '.join(s), axis=1)

# fit TfidfVectorizer with all sites from user sessions
vec = TfidfVectorizer(ngram_range=(1, 3), max_features=200000, stop_words=['0'])
vec = vec.fit(test_sessions.append(train_sessions))

# generate sparse matrices
train_v = vec.transform(train_sessions)
test_v = vec.transform(test_sessions)

CPU times: user 1min 14s, sys: 1.31 s, total: 1min 16s
Wall time: 1min 16s


Идеи:
* Пользователи предпочитают пользоваться определённым поисковиком и определённой соцсетью, таким образом можно ввести категориальные признаки: поисковик, соцсеть со значениями (google, yandex, mail.ru, rambler, microsoft ... и facebook, vk, odnoklassniki ...)
* Продолжительность пребывания на сайте может нести полезную информацию, поэтому разреженная матрица, где в строках сессии, а в столбцах сайты, со значением время пребывания за сессию может улучшить модель теоретически 
* Возможно сработают комбинации типа "любимые сайты по утрам" или "любимые сайты по выходным"

Add features based on the session start time: hour, whether it's morning, day or night and so on.
Scale this features and combine then with Tf-Idf based on sites (you'll need `scipy.sparse.hstack`)

In [12]:
cat_features = ['hour_of_day', 'part_of_day', 'day_of_week', 'weekend']
scalable_features = ['session_span']
other_features = ['#unique_sites']
all_features = cat_features + scalable_features + other_features

features_train = train_dataset[all_features]
features_test = test_dataset[all_features]

scaler = MinMaxScaler()
features_train[scalable_features] = scaler.fit_transform(features_train[scalable_features])
features_test[scalable_features] = scaler.transform(features_test[scalable_features])

In [13]:
enc = LabelEncoder()
enc.fit(cat_features)
new_cat_features = enc.transform(cat_features)

encoder = OneHotEncoder(categorical_features=new_cat_features)
train_mtx = encoder.fit_transform(features_train)
test_mtx = encoder.transform(features_test)

In [14]:
train_X = hstack([train_mtx, train_v])
test_X = hstack([test_mtx, test_v])

Perform cross-validation with logistic regression.

In [15]:
%%time
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
logitCV = LogisticRegressionCV(
        Cs=10,  # [1, 11.55, 12.0, 13.27],
        penalty='l2',
        scoring='roc_auc',
        cv=skf,
        random_state=42,
        solver='liblinear',
        n_jobs=-1,
        refit=True,
        verbose=2,
        max_iter=100,
        tol=0.0001
    )

logitCV.fit(train_X, y)


os.system('say "your program has finished"')

[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 22.5min finished


[LibLinear]CPU times: user 24.3 s, sys: 2.58 s, total: 26.9 s
Wall time: 22min 52s


Let's evaluate our model:

In [16]:
print ('AUC_ROC for our model:', logitCV.scores_[1].mean(axis=0).max())

AUC_ROC for our model: 0.988053446221


Experiments:

 * TFxIDF+hour,pod,dow,is_weekend,session_span,#unique sites /np.logspace(-7, 7, 15); l1; 3 folds/ = 0.98138
 * TFxIDF+hour,pod,dow,is_weekend,session_span,#unique sites /np.logspace(-2, 3, 9); l2; 3 folds/ = 0.98430
 * TFxIDF+hour,pod,dow,is_weekend,session_span,#unique sites /l2; 3 folds; ngram_range=(1,2), C=12.55/ = 0.98446
 * TFxIDF only /.../ = 0.95835
 * TFxIDF+hour,pod,dow,is_weekend,session_span,#unique sites /l2; 5 folds; ngram_range=(1,3), C=11.55/ = 0.98626403243
 * ... /10 folds/ = 0.98667
 * ... /tol=0.00001/ = 0.98667
 * ...+avg_per_site = 0.98668
 * ...+TFxIDF(max_features=200000) = 0.98671
 * ...+TFxIDF(max_features=500000) = 0.98598
 * tuned hour feature+TFxIDF(max_features=200000)  = 0.98671
 * ...+day,month,year = 0.99170 (probably overfit 0.90 on kaggle)
 * ...+month = 0.98994
 * TFxIDF+hour,pod,dow,is_weekend,session_span,#unique sites, tuned hour, avg_per_site/l2; 10 folds; ngram_range=(1,3), C=12./ = 0.98807
 * ... C=13.27 = 0.98808

In [17]:
logitCV.C_

array([ 21.5443469])

Make prediction for the test set and form a submission file.

In [18]:
test_pred = logitCV.predict_proba(test_X)[:,1]
test_pred = np.array(list("{:.6f}".format(x) for x in test_pred))

In [19]:
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)


In [20]:
write_to_submission_file(test_pred, "assignment6_alice_submission.csv")